In [28]:
from transformers.utils import logging
logging.set_verbosity_error()

In [50]:
# Use a pipeline as a high-level helper
from transformers import pipeline, Conversation

chatbot = pipeline("text2text-generation", model="facebook/blenderbot-400M-distill")


user_message = "What are some fun activities I can do in the winter?"
conversation = Conversation(user_message)
print(conversation)

Conversation id: 7b4644d8-22ce-463a-93fb-8774dd54fafd
user: What are some fun activities I can do in the winter?



In [53]:
conversation = chatbot(conversation)

print(conversation)


conversation.add_message(
    {"role": "user",
     "content": """
What else do you recommend?
"""
    })

print(conversation)

ValueError:  `args[0]`: Conversation id: 7b4644d8-22ce-463a-93fb-8774dd54fafd
user: What are some fun activities I can do in the winter?
 have the wrong format. The should be either of type `str` or type `list`

Translation and Summarization

In [56]:
from transformers import pipeline 
import torch

translator = pipeline(task="translation",
                      model="facebook/nllb-200-distilled-600M",
                      torch_dtype=torch.bfloat16)


Error while downloading from https://cdn-lfs.huggingface.co/repos/6e/ef/6eef128ace768b50a05cfccb07249581231dafe7b91ae38b6d142cbc66dab00d/c266c2cfd19758b6d09c1fc31ecdf1e485509035f6b51dfe84f1ada83eefcc42?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1714918702&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDkxODcwMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82ZS9lZi82ZWVmMTI4YWNlNzY4YjUwYTA1Y2ZjY2IwNzI0OTU4MTIzMWRhZmU3YjkxYWUzOGI2ZDE0MmNiYzY2ZGFiMDBkL2MyNjZjMmNmZDE5NzU4YjZkMDljMWZjMzFlY2RmMWU0ODU1MDkwMzVmNmI1MWRmZTg0ZjFhZGE4M2VlZmNjNDI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=zUUmaRu-2peEKsc6xR-SzC5Smfh55zxSPZTOwprI0OEqok8FwTqiMI1Ox5R2bLRO7qzbeBfUVz6Pu2GMfVABJoSDosnWgClNEHiL6PTo77WBjkxDdyKnQg7vvPw7TXzjpAlOCY4WPhsQCGQrCJQu5LM2dbKlNke6j

In [ ]:
text = """\
My puppy is adorable, \
Your kitten is cute.
Her panda is friendly.
His llama is thoughtful. \
We all have nice pets!"""

text_translated = translator(text,
                             src_lang="eng_Latn",
                             tgt_lang="fra_Latn")
